#### After testing multiple ways, two possible solutions were chosen.
#### The first solution is by using a python library, seleniumbase along with proxies. This solution works well if there are some premium / elite proxies available. Since the ones I used are free proxies, they were getting blocked pretty easily after like 2-3 attempts.

#### The second solution was to use the already available API which can bypass the bot detection. I have used the free version of it.

# Solution using seleniumbase

In [ ]:
# importing the required libraries

import pandas as pd
import bs4
import numpy as np
from datetime import datetime
from seleniumbase import SB
import random
from selenium.webdriver.common.by import By
import bs4
from tqdm import tqdm

In [ ]:
#few popular user_agents for rotating headers to avoid getting blocked

user_agents = [    
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.81",
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 14.4; rv:124.0) Gecko/20100101 Firefox/124.0",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
"Mozilla/5.0 (X11; Linux i686; rv:124.0) Gecko/20100101 Firefox/124.0"
 ]

In [ ]:
#list of proxies

proxies = ["scraperapi:034fe3480dc150876e289195587f76a4@proxy-server.scraperapi.com:8001",
           "idlitvub:yubsxgfsk4ku@45.94.47.66:8110",
           "idlitvub:yubsxgfsk4ku@185.199.229.156:7492",
           "idlitvub:yubsxgfsk4ku@185.199.228.220:7300",
           "idlitvub:yubsxgfsk4ku@185.199.231.45:8382",
           "idlitvub:yubsxgfsk4ku@188.74.210.207:6286",
           "idlitvub:yubsxgfsk4ku@188.74.210.21:6100",
           "idlitvub:yubsxgfsk4ku@154.95.36.199:6893",
           "idlitvub:yubsxgfsk4ku@45.155.68.129:8133",
           "idlitvub:yubsxgfsk4ku@188.74.183.10:8279"
           ]

In [ ]:
# Reading the input file for isbn13
inp = pd.read_csv("input_list.csv")

# Initialising an empty dataframe with all the required fields
df = pd.DataFrame(columns=["ISBN 13", "Title of the Book", "Author/s", "Book type", "Original Price (RRP)($)", "Discounted price($)",
                          "ISBN-10", "Published Date", "Publisher", "No. of Pages"])

In [ ]:
# Function to convert the published date to the required format(YYYY-MM-DD)

def date_string_to_date(date_str):
    # Remove the ordinal indicator
    date_str = date_str.replace("st", "").replace("nd", "").replace("rd", "").replace("th", "")

    # Parse the string into a datetime object
    date_obj = datetime.strptime(date_str, "%d %B %Y")

    # Format the datetime object into "YYYY-MM-DD"
    formatted_date = date_obj.strftime("%Y-%m-%d")
    return formatted_date

In [ ]:
# Scraping the required fields

j = 0
proxy_index = 0

for isbn in tqdm(range(len(inp['ISBN13']))):
    random_user_agent = random.choice(user_agents)
    current_proxy = proxies[proxy_index]
    
    #creating an instance of selenium driver
    with SB(
            browser="chrome",
            demo=True,
            uc=True,
            headless2=False,
            demo_sleep=3,       
            agent=random_user_agent,
            do_not_track=True,
            undetectable=True,
            proxy=current_proxy,
            headless=True,
        ) as sb:

        url = f"https://www.booktopia.com.au/ebook/{inp['ISBN13'][isbn]}.html"

        sb.driver.uc_open(url) #opens url in undetected mode
        sb.driver.delete_all_cookies()#deleting any cookies if saved


        sb.sleep(4)
        soup = bs4.BeautifulSoup(sb.driver.page_source, "html.parser")

        # Checking if the book exists on the website
        try:  
            title = soup.find("h1", class_="MuiTypography-root MuiTypography-h1 mui-style-1ngtbwk").text
        except:
            df.loc[j,"ISBN 13"] =inp["ISBN13"][isbn]
            df.loc[j, "Title of the Book"] = "book not found"
            j += 1
            continue

        df.loc[j, "ISBN 13"] = inp['ISBN13'][isbn]
        df.loc[j, "Title of the Book"] = title
        author = soup.find("span", class_="MuiTypography-root MuiTypography-body1 mui-style-1plnxgp").text
        df.loc[j, "Author/s"] = author
        details = (soup.find("div", class_="MuiBox-root mui-style-h3npb")).find_all("p", "MuiTypography-root MuiTypography-body1 mui-style-tgrox")
        
        # Getting the price of the book if it exists
        try:
            try:  
                price = soup.find("div", class_="MuiStack-root mui-style-cfla3").text
                p = price.split("$")
                try:
                    if "RRP" in price:
                        df.loc[j, "Discounted price($)"] = float(p[2])
                        df.loc[j, "Original Price (RRP)($)"] = float(p[1])
                    else:
                        df.loc[j, "Original Price (RRP)($)"] = float(p[1])
                except:
                    if "RRP" in price:
                        df.loc[j, "Discounted price($)"] = (p[2])
                        df.loc[j, "Original Price (RRP)($)"] = (p[1])
                    else:
                        df.loc[j, "Original Price (RRP)($)"] = (p[1])   
            except:
                price=(soup.find("button",class_="MuiButtonBase-root MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeLarge MuiButton-outlinedSizeLarge MuiButton-fullWidth Mui-disabled MuiButtonGroup-grouped MuiButtonGroup-groupedHorizontal MuiButtonGroup-groupedOutlined MuiButtonGroup-groupedOutlinedHorizontal MuiButtonGroup-groupedOutlinedPrimary MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeLarge MuiButton-outlinedSizeLarge MuiButton-fullWidth BestRelatedProducts_related-product__TJBt9 BestRelatedProducts_current-product__donyl false mui-style-r6fh0t")).find("p","MuiTypography-root MuiTypography-body1 mui-style-tgrox").text
                df.loc[j, "Original Price (RRP)($)"] = (price)
#                 print(price)       
        except:
            pass
        
        # Getting other details like publisher, number of pages etc.
        book_details = {}
        for i in details:
            i = i.text.split(":")
            book_details[i[0]] = i[1].strip()
        
    
        df.loc[j, "Book type"] = book_details.get("Format")
        if book_details.get("ISBN-10") != None:
            try:
                df.loc[j, "ISBN-10"] = np.int64(book_details.get("ISBN-10"))
            except:
                df.loc[j, "ISBN-10"] = (book_details.get("ISBN-10"))
                
        
        if book_details.get("Published") != None:
            try:
                df.loc[j, "Published Date"] = date_string_to_date(book_details.get("Published"))
            except:
                df.loc[j, "Published Date"] = (book_details.get("Published"))
                
        
        df.loc[j, "Publisher"] = book_details.get("Publisher")
        if (book_details.get("Number of Pages")) != None:
             try:
                df.loc[j, "No. of Pages"] = int(book_details.get("Number of Pages"))
            except:
                df.loc[j, "No. of Pages"] = (book_details.get("Number of Pages"))
    
        j+= 1
        
        # Saving the file after every 15 records   
        if j % 15 == 0:
            df.to_csv("book_details.csv", index=False)
        
        proxy_index = (proxy_index + 1) % len(proxies) # roatiton of proxies on by one
        sb.sleep(12) # as per the website, we have crawl delay of 10 seconds.

df.to_csv("book_details.csv", index=False)

# Solution using available API (zenrows)

In [ ]:
# importing the required libraries

import datetime
from zenrows import ZenRowsClient
import pandas as pd
import bs4
import numpy as np
from tqdm import tqdm
import time

In [ ]:
# Initialising zenrows client. Usually this detail is added in .env file

client = ZenRowsClient("c9ce38ebf34affe06bb61ff04ec8c7181565") # I have changed the value of it for privacy pupose
params = {"premium_proxy":"true","proxy_country":"au"}

In [ ]:
# Reading the input file for isbn13
inp = pd.read_csv("input_list.csv")

# Initialising an empty dataframe with all the required fields
df = pd.DataFrame(columns=["ISBN 13", "Title of the Book", "Author/s", "Book type", "Original Price (RRP)($)", "Discounted price($)",
                          "ISBN-10", "Published Date", "Publisher", "No. of Pages"])

In [ ]:
# Function to convert the published date to the required format(YYYY-MM-DD)

def date_string_to_date(date_str):
    # Remove the ordinal indicator
    date_str = date_str.replace("st", "").replace("nd", "").replace("rd", "").replace("th", "")

    # Parse the string into a datetime object
    date_obj = datetime.strptime(date_str, "%d %B %Y")

    # Format the datetime object into "YYYY-MM-DD"
    formatted_date = date_obj.strftime("%Y-%m-%d")

In [ ]:
j = 0
for isbn in tqdm(range(len(inp['ISBN13']))):
    url = f"https://www.booktopia.com.au/ebook/{inp['ISBN13'][isbn]}.html"
    response = client.get(url, params=params)

    soup = bs4.BeautifulSoup(response.content, "html.parser")
    
    # Checking if the book exists on the website
    try:  
        title = soup.find("h1", class_="MuiTypography-root MuiTypography-h1 mui-style-1ngtbwk").text
    except:
        df.loc[j,"ISBN 13"] =inp["ISBN13"][isbn]
        df.loc[j, "Title of the Book"] = "book not found"
        j += 1
        continue

    df.loc[j, "ISBN 13"] = inp['ISBN13'][isbn]
    df.loc[j, "Title of the Book"] = title
    author = soup.find("span", class_="MuiTypography-root MuiTypography-body1 mui-style-1plnxgp").text
    df.loc[j, "Author/s"] = author
    details = (soup.find("div", class_="MuiBox-root mui-style-h3npb")).find_all("p", "MuiTypography-root MuiTypography-body1 mui-style-tgrox")
    
    # Getting the price of the book if it exists
    try:
        try:  
            price = soup.find("div", class_="MuiStack-root mui-style-cfla3").text
            p = price.split("$")
            try:
                if "RRP" in price:
                    df.loc[j, "Discounted price($)"] = float(p[2])
                    df.loc[j, "Original Price (RRP)($)"] = float(p[1])
                else:
                    df.loc[j, "Original Price (RRP)($)"] = float(p[1])
            except:
                if "RRP" in price:
                    df.loc[j, "Discounted price($)"] = (p[2])
                    df.loc[j, "Original Price (RRP)($)"] = (p[1])
                else:
                    df.loc[j, "Original Price (RRP)($)"] = (p[1])   
        except:
            try:
                price=(soup.find("button",class_="MuiButtonBase-root MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeLarge MuiButton-outlinedSizeLarge MuiButton-fullWidth Mui-disabled MuiButtonGroup-grouped MuiButtonGroup-groupedHorizontal MuiButtonGroup-groupedOutlined MuiButtonGroup-groupedOutlinedHorizontal MuiButtonGroup-groupedOutlinedPrimary MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeLarge MuiButton-outlinedSizeLarge MuiButton-fullWidth BestRelatedProducts_related-product__TJBt9 BestRelatedProducts_current-product__donyl false mui-style-r6fh0t")).find("p","MuiTypography-root MuiTypography-body1 mui-style-tgrox").text
                p = price.split("$")
                df.loc[j, "Original Price (RRP)($)"] = float(p[1])
                print(price)      
            except:
                df.loc[j, "Original Price (RRP)($)"] = (p[1])

    except:
        pass
        
    # Getting other details like publisher, number of pages etc.
    book_details = {}
    for i in details:
        i = i.text.split(":")
        book_details[i[0]] = i[1].strip()
       
   
    df.loc[j, "Book type"] = book_details.get("Format")

    if book_details.get("ISBN-10") != None:
        try:
            df.loc[j, "ISBN-10"] = np.int64(book_details.get("ISBN-10"))
        except:
            df.loc[j, "ISBN-10"] = (book_details.get("ISBN-10"))
       
    if book_details.get("Published") != None:
        try:
            df.loc[j, "Published Date"] = date_string_to_date(book_details.get("Published"))
        except:
            df.loc[j, "Published Date"] = (book_details.get("Published"))
       
    df.loc[j, "Publisher"] = book_details.get("Publisher")
    if (book_details.get("Number of Pages")) != None:
        try:
            df.loc[j, "No. of Pages"] = int(book_details.get("Number of Pages"))
        except:
            df.loc[j, "No. of Pages"] = (book_details.get("Number of Pages"))
   
    j+= 1
   
    # Saving the file after every 15 records
    if j % 15 == 0:
        df.to_csv("bookDetails.csv", index=False)

df.to_csv("bookDetails.csv", index=False)